## Prompt + LLM

- **Key Composition in LangChain**:
  - The most frequent and effective composition in LangChain involves the sequence: 
    - `PromptTemplate` or `ChatPromptTemplate`
    - Followed by `LLM` or `ChatModel`
    - And then an `OutputParser`.
  - This foundational building block forms the basis of most other chains constructed in LangChain.

### PromptTemplate + LLM

- **Basic Composition in LangChain**:
  - The simplest form of composition in LangChain involves combining a prompt with a model.
  - This basic chain process includes:
    - Taking user input.
    - Incorporating it into a prompt.
    - Passing the prompt to a model.
    - Returning the raw output from the model.
  - Flexibility is offered in this composition, allowing for mixing and matching of `PromptTemplate`/`ChatPromptTemplate` with `LLM`/`ChatModel` as needed.


In [1]:
from langchain.chat_models.openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("tell me a joke about {foo}")
model = ChatOpenAI()
chain = prompt | model

In [2]:
chain.invoke(
  {
    "foo": "bears"
  }
)

AIMessage(content="Sure, here's a bear joke for you:\n\nWhy don't bears like fast food?\n\nBecause they can't catch it!")

Often times we want to attach kwargs that’ll be passed to each model call. Here are a few examples of that:

### Attaching Stop Sequences

In [5]:
chain = prompt | model.bind(stop=["\n"])
chain.invoke(
  {
    "foo": "bears"
  }
)

AIMessage(content="Why don't bears wear socks? ")

### Attaching Function Call information

In [6]:
functions = [
    {
        "name": "joke",
        "description": "A joke",
        "parameters": {
            "type": "object",
            "properties": {
                "setup": {"type": "string", "description": "The setup for the joke"},
                "punchline": {
                    "type": "string",
                    "description": "The punchline for the joke",
                },
            },
            "required": ["setup", "punchline"],
        },
    }
]


In [7]:
chain = prompt | model.bind(function_call={"name": "joke"}, functions=functions)

In [8]:
chain.invoke({"foo": "bears"})

AIMessage(content='', additional_kwargs={'function_call': {'name': 'joke', 'arguments': '{\n  "setup": "Why don\'t bears wear shoes?",\n  "punchline": "Because they have bear feet!"\n}'}})

### Functions Output Parser

When you specify the function to return, you may just want to parse that directly



In [27]:
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

chain = (
    prompt
    | model.bind(function_call={"name": "joke"}, functions=functions)
    | JsonOutputFunctionsParser()
)

In [28]:
chain.invoke({"foo": "bears"})

{'setup': "Why don't bears wear shoes?",
 'punchline': 'Because they have bear feet!'}

In [29]:
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser

chain = (
    prompt
    | model.bind(function_call={"name": "joke"}, functions=functions)
    | JsonKeyOutputFunctionsParser(key_name="setup")
)

chain.invoke({"foo": "bears"})

"Why don't bears wear shoes?"

### Simplifying input

To make invocation even simpler, we can add a RunnableParallel to take care of creating the prompt input dict for us:

In [22]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough


In [ ]:
map_ = RunnableParallel(foo=RunnablePassthrough())

chain = (
    map_
    | prompt
    | model.bind(function_call={"name": "joke"}, functions=functions)
    | JsonKeyOutputFunctionsParser(key_name="setup")
)